### This notebook consists of build LSTM model on the data for COVID-19 Forecasting Week 4 .  
#### The data consists of count of confirmed cases , number of fatalties recorded on dailly basis for 180 countries over a period of 72 days - 22nd Jan 2020 - 22nd Mar 2020. 
#### The ask is develop a predictive model which would forecast the number of covid cases and fatalities over the next 43 days for these 180 countries. For some countries, the data is also recorded for its indivual states, thus total number of regions - 313. 

#### The Bi-Directional LSTM model is trained to predict for next 43 steps for previous 29 steps. 
#### The timeseries are clustered into 10 groups using the K-Means algorithms  

In [2]:
import numpy as np
import pandas as pd
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

#### The below two funtions written to formulate the training data for confirmed cases and fatalities

In [10]:
def train_confirmed_cases(data, n_input, n_out):
    # flatten data
    #data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        #define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end, :]
            x_input = x_input.reshape((len(x_input), 11))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return np.array(X), np.array(y)

In [11]:
### Fatalities
def train_fatalities(data, n_input, n_out):
    # flatten data
    #data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        #define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end, :]
            x_input = x_input.reshape((len(x_input), 11))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
    return np.array(X), np.array(y)

#### The below fuuction is used to formulate the test data for predictions

In [12]:
def get_test_cc(data, n_input, n_out):
    Xtest = list()
    in_start = 0
    #for _ in range(len(data)):
    in_end = in_start + n_input
    #out_end = in_end + n_out
    #if out_end <= len(data):
    x_input = data[in_start:in_end, :]
    x_input = x_input.reshape((len(x_input), 11))
    Xtest.append(x_input)
        #in_start += 1
    return np.array(Xtest)
def get_test_ft(data, n_input, n_out):
    Xtest = list()
    in_start = 0
    #for _ in range(len(data)):
    in_end = in_start + n_input
    #out_end = in_end + n_out
    #if out_end <= len(data):
    x_input = data[in_start:in_end, :]
    x_input = x_input.reshape((len(x_input), 11))
    Xtest.append(x_input)
        #in_start += 1
    return np.array(Xtest)

### Read the Train Data

#### The file 'Train_Data_Group.csv' is a modified file which has been created out of the first notebook 

In [13]:
df = pd.read_csv('Train_Data_Group.csv',sep='|')
country_state_province_list=list(set(list(df['Country_state_Province'])))
#df[df['Country_state_Province']=='US-Minnesota']
df2=pd.get_dummies(df, columns=['Group_Num'])

In [14]:
cols_list=['ConfirmedCases','Fatalities','Country_state_Province','Group_Num_0','Group_Num_1','Group_Num_2','Group_Num_3','Group_Num_4','Group_Num_5','Group_Num_6','Group_Num_7','Group_Num_8','Group_Num_9']
df3=df2[cols_list]

In [15]:
X_train=[]
y_train=[]
X_train_ft=[]
y_train_ft=[]

In [16]:
for region in country_state_province_list:
    dfregion=df3[df3['Country_state_Province']==region]
    dfregion_vals_cc=dfregion.drop(['Country_state_Province','Fatalities'], axis=1).values
    dfregion_vals_ft=dfregion.drop(['Country_state_Province','ConfirmedCases'], axis=1).values
    x_train_row, y_train_row=train_confirmed_cases(dfregion_vals_cc, 29, 43)
    x_train_row_ft, y_train_row_ft=train_fatalities(dfregion_vals_ft, 29, 43)
    X_train.append(x_train_row)
    y_train.append(y_train_row)
    X_train_ft.append(x_train_row_ft)
    y_train_ft.append(y_train_row_ft)

In [17]:
X_train_arr=np.array(X_train)
y_train_arr=np.array(y_train)
X_train_ft_arr=np.array(X_train_ft)
y_train_ft_arr=np.array(y_train_ft)
print(X_train_arr.shape)
print(y_train_arr.shape)
print(X_train_ft_arr.shape)
print(y_train_ft_arr.shape)

(313, 8, 29, 11)
(313, 8, 43)
(313, 8, 29, 11)
(313, 8, 43)


In [18]:
X_train_arr=np.array(X_train).reshape(X_train_arr.shape[0]*X_train_arr.shape[1],29,11)
y_train_arr=np.array(y_train).reshape(y_train_arr.shape[0]*y_train_arr.shape[1],43)
X_train_ft_arr=np.array(X_train_ft).reshape(X_train_ft_arr.shape[0]*X_train_ft_arr.shape[1],29,11)
y_train_ft_arr=np.array(y_train_ft).reshape(y_train_ft_arr.shape[0]*y_train_ft_arr.shape[1],43)
print(X_train_arr.shape)
print(y_train_arr.shape)
print(X_train_ft_arr.shape)
print(y_train_ft_arr.shape)

(2504, 29, 11)
(2504, 43)
(2504, 29, 11)
(2504, 43)


In [19]:
y_train_arr=y_train_arr.reshape(2504,1,43)
y_train_ft_arr=y_train_ft_arr.reshape(2504,1,43)

In [20]:
y_train_arr

array([[[  94.,  105.,  122., ..., 1953., 2178., 2495.]],

       [[ 105.,  122.,  147., ..., 2178., 2495., 2617.]],

       [[ 122.,  147.,  159., ..., 2495., 2617., 3139.]],

       ...,

       [[ 576.,  576.,  576., ...,  579.,  579.,  579.]],

       [[ 576.,  576.,  576., ...,  579.,  579.,  579.]],

       [[ 576.,  576.,  576., ...,  579.,  579.,  579.]]])

#### Standardize dataset

In [21]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
scy=StandardScaler()
sfx=StandardScaler()
sfy=StandardScaler()
X_train_arr_scaled=scx.fit_transform(X_train_arr.reshape(2504,29*11)).reshape(2504, 29, 11)
y_train_arr_scaled=scy.fit_transform(y_train_arr.reshape(2504,43)).reshape(2504,1,43)
X_train_ft_arr_scaled=sfx.fit_transform(X_train_ft_arr.reshape(2504,29*11)).reshape(2504, 29, 11)
y_train_ft_arr_scaled=sfy.fit_transform(y_train_ft_arr.reshape(2504,43)).reshape(2504,1,43)

### Model Training

#### Model to Train Confirmed Cases

In [69]:
from keras.callbacks import ModelCheckpoint
from keras.layers import RepeatVector,TimeDistributed
filepath="cc_model_weights_1_input_approach_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [81]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(29, 11)))
model.add(RepeatVector(1))
model.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model.add(TimeDistributed(Dense(43)))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train_arr_scaled, y_train_arr_scaled, epochs=50, validation_split=0.1, verbose=1, batch_size=50,callbacks=callbacks_list)

Train on 2253 samples, validate on 251 samples
Epoch 1/50
2253/2253 [==============================] - 11s 5ms/step - loss: 0.8346 - val_loss: 0.0216
Epoch 2/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2670 - val_loss: 0.0133
Epoch 3/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.2044 - val_loss: 0.0086
Epoch 4/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.1528 - val_loss: 0.0123
Epoch 5/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.1052 - val_loss: 0.0087
Epoch 6/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.0901 - val_loss: 0.0081
Epoch 7/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.0658 - val_loss: 0.0089
Epoch 8/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.0735 - val_loss: 0.0085
Epoch 9/50
2253/2253 [==============================] - 4s 2ms/step - loss: 0.0867 - val_loss: 0.0095
Epoch 10/50
2253/2253 [===========

#### Model to Train Fatalities

In [119]:
from keras.callbacks import ModelCheckpoint
filepath="ft_model_weights_1_input_approach_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [121]:
model1 = Sequential()
model1.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(29, 11)))
model1.add(RepeatVector(1))
model1.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True)))
model1.add(TimeDistributed(Dense(43)))
model1.compile(optimizer='adam', loss='mse')

history = model1.fit(X_train_ft_arr_scaled, y_train_ft_arr_scaled, epochs=20, validation_split=0.1, verbose=1, batch_size=50,callbacks=callbacks_list)

Train on 2253 samples, validate on 251 samples
Epoch 1/20
2253/2253 [==============================] - 7s 3ms/step - loss: 1.4513 - val_loss: 0.0070
Epoch 2/20
2253/2253 [==============================] - 3s 2ms/step - loss: 3.4481 - val_loss: 0.0032
Epoch 3/20
2253/2253 [==============================] - 3s 2ms/step - loss: 1368.7576 - val_loss: 0.0065
Epoch 4/20
2253/2253 [==============================] - 4s 2ms/step - loss: 46.9273 - val_loss: 0.0030
Epoch 5/20
2253/2253 [==============================] - 4s 2ms/step - loss: 18.2414 - val_loss: 0.0017
Epoch 6/20
2253/2253 [==============================] - 4s 2ms/step - loss: 35.6411 - val_loss: 0.0017
Epoch 7/20
2253/2253 [==============================] - 3s 2ms/step - loss: 6.8277 - val_loss: 0.0018
Epoch 8/20
2253/2253 [==============================] - 5s 2ms/step - loss: 1.2902 - val_loss: 0.0017
Epoch 9/20
2253/2253 [==============================] - 4s 2ms/step - loss: 1.3597 - val_loss: 0.0017
Epoch 10/20
2253/2253 [======

### Model Prediction

#### Formulate the data for prediction

In [22]:
cols_list=['ConfirmedCases','Fatalities','Country_state_Province','Group_Num_0','Group_Num_1','Group_Num_2','Group_Num_3','Group_Num_4','Group_Num_5','Group_Num_6','Group_Num_7','Group_Num_8','Group_Num_9']
dftest_all_cols=df2[df2['Day']>50]
dftest=dftest_all_cols[cols_list]

In [23]:
X_test_CC=[]
X_test_FT=[]
for region in country_state_province_list:
    dfregion=dftest[dftest['Country_state_Province']==region]
    dfregion_vals_cc=dfregion.drop(['Country_state_Province','Fatalities'], axis=1).values
    dfregion_vals_ft=dfregion.drop(['Country_state_Province','ConfirmedCases'], axis=1).values
    x_test_row_cc=get_test_cc(dfregion_vals_cc, 29, 43)
    x_test_row_ft=get_test_ft(dfregion_vals_ft, 29, 43)
    #x_train_row_ft, y_train_row_ft=to_supervised_fatalities(dfregi
    X_test_CC.append(x_test_row_cc)
    X_test_FT.append(x_test_row_ft)
X_test_cc_arr=np.array(X_test_CC)
X_test_ft_arr=np.array(X_test_FT)
print(X_test_cc_arr.shape)
print(X_test_ft_arr.shape)
X_test_cc_arr=np.array(X_test_CC).reshape(X_test_cc_arr.shape[0]*X_test_cc_arr.shape[1],29,11)
X_test_ft_arr=np.array(X_test_FT).reshape(X_test_ft_arr.shape[0]*X_test_ft_arr.shape[1],29,11)
print(X_test_cc_arr.shape)
X_test_cc_arr_scaled=scx.transform(X_test_cc_arr.reshape(313,29*11)).reshape(313, 29, 11)
X_test_ff_arr_scaled=sfx.transform(X_test_ft_arr.reshape(313,29*11)).reshape(313, 29, 11)

(313, 1, 29, 11)
(313, 1, 29, 11)
(313, 29, 11)


#### Predict the Data Using the two trained models

In [122]:
y_cc_pred=model.predict(X_test_cc_arr_scaled)
y_ft_pred=model1.predict(X_test_ff_arr_scaled)

In [124]:
y_cc_pred_inv_out=scy.inverse_transform(y_cc_pred.reshape(313,43))
y_ft_pred_inv_out=sfy.inverse_transform(y_ft_pred.reshape(313,43))

In [126]:
dte_list=[]
import datetime

start = datetime.datetime.strptime("2020-04-02", "%Y-%m-%d")
end = datetime.datetime.strptime("2020-05-15", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    dte_list.append(date.strftime("%Y-%m-%d"))

In [127]:
day_list=[]
for i in range(1,44):
    day_list.append('Day_' +str(i))

In [128]:
dresult_base=pd.DataFrame(data=day_list, index=range(len(day_list)), columns=['Day'])
dresult_base['Predict_Date']=dte_list
dresult_base.head()


,Day,Predict_Date
0,Day_1,2020-04-02
1,Day_2,2020-04-03
2,Day_3,2020-04-04
3,Day_4,2020-04-05
4,Day_5,2020-04-06


In [129]:
dresult_out=pd.DataFrame(columns=['Day','Predict_Date','Region','Predicted_Confirmed_Cases','Predicted_Fatalities'])
for j in range(len(country_state_province_list)):
    dresult_region=dresult_base
    dresult_region['Region']=country_state_province_list[j]
    dresult_region['Predicted_Confirmed_Cases']=y_cc_pred_inv_out[j]
    dresult_region['Predicted_Fatalities']=y_ft_pred_inv_out[j]
    dresult_out=dresult_out.append(dresult_region)
    dresult_out.index=range(len(dresult_out))

In [131]:
dresult_out.to_csv('Coronavirus_Forecast_LSTM_Model_Week_4.csv',sep='|')

#### Merging the Result with the Test file for submission

In [132]:
dtest_input=pd.read_csv('test.csv')
dtest_input.head()

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-04-02
1,2,NaN,Afghanistan,2020-04-03
2,3,NaN,Afghanistan,2020-04-04
3,4,NaN,Afghanistan,2020-04-05
4,5,NaN,Afghanistan,2020-04-06


In [133]:
dtest_input['Province_State'].fillna('NA', inplace=True)
dtest_input['Country_Region_Province']=dtest_input['Country_Region']+'-'+dtest_input['Province_State']

In [134]:
dresult_out.head()

,Day,Predict_Date,Region,Predicted_Confirmed_Cases,Predicted_Fatalities
0,Day_1,2020-04-02,China-Hebei,436.487518,10.074240
1,Day_2,2020-04-03,China-Hebei,357.098358,16.962048
2,Day_3,2020-04-04,China-Hebei,345.330658,18.204575
3,Day_4,2020-04-05,China-Hebei,626.100159,15.599738
4,Day_5,2020-04-06,China-Hebei,396.978729,14.349257


In [135]:
d_result=pd.merge(dresult_out,dtest_input, how='inner',left_on=['Region','Predict_Date'], right_on=['Country_Region_Province','Date'])

In [136]:
kaggle_out_df=d_result[['ForecastId','Predicted_Confirmed_Cases','Predicted_Fatalities']]
kaggle_out_df.sort_values(by=['ForecastId'],inplace=True)
kaggle_out_df.to_csv('submission.csv',index=None)

c:\users\ravi_\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Dumping the objects

In [137]:
import joblib
joblib.dump(scx,'Scaler_Confirmed_Cases_X.pkl')
joblib.dump(scy,'Scaler_Confirmed_Cases_Y.pkl')
joblib.dump(sfx,'Scaler_Fatalities_X.pkl')
joblib.dump(sfy,'Scaler_Fatalities_Y.pkl')

['Scaler_Fatalities_Y.pkl']